In [1]:
wd <- dirname(dirname(getwd()))
source(paste0(wd,"/mission_control/treasure_map.R"))
library(tidyverse)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()  

### 0 - Read individual studies 

In [55]:
ready <- readRDS(paste0(TMP_DIR,'exhaustive-plots-base.Rds'))

### 1 - Summarise

In [79]:
k <- .3

In [80]:
(ready %>% filter(model == "bor", covariates == "age_biopsy_purity_tissue", p_val < by_05_fdr, 
                 abs(cor_tcell) < k, abs(cor_tmb) < k, abs(cor_tgfb) < k, abs(cor_prolif) < k, abs(cor_pretreat) < k)
      %>% select(feature, est, p_val, contains("cor"), by_05_fdr))

feature,est,p_val,cor_pretreat,cor_tmb,cor_tcell,cor_prolif,cor_tgfb,cor_purity,by_05_fdr
isofox_BAIAP2,-0.5402634,3.848490e-05,-0.07072406,-0.002574333,-0.15063613,-0.05134054,-0.11047526,0.07285417,5.534163e-05
isofox_LATS2,-0.6313075,1.468607e-05,-0.15024418,0.030620150,0.05548550,-0.10944140,0.26614789,-0.08569350,5.534163e-05
somatic.gene_FCGR2B.mb,0.4663670,3.376074e-05,0.04563883,0.265085141,0.07489015,0.11523019,0.04141224,-0.03183556,5.534163e-05


In [56]:
go <- ready %>% select(dataset, Type, feature, covariates, model, p_val, contains("_05_"))

- Total counts

In [57]:
see <- go %>% mutate(model2 = ifelse( model == "os" & covariates == "residuals", "os_resid", model))

In [37]:
#head(go)

In [45]:
(
 go 
   %>% mutate(model = ifelse( model == "os" & covariates == "residuals", "os_resid", model))
   %>% mutate(covariates = ifelse( model == "os_resid" & covariates == "residuals", "age_biopsy_purity_tissue", as.character(covariates)))
   %>% filter(dataset == "all", covariates == "age_biopsy_purity_tissue")
   %>% group_by(Type, model)
   %>% mutate(ct = n(), 
                 bh = p_val < bh_05_fdr,
                 by = p_val < by_05_fdr,
                 bf = p_val < bf_05_fwe)
   %>% group_by(Type, feature)
   %>% summarise( bh = sum(bh) > 0, by = sum(by) > 0, bf = sum(bf) > 0)
   %>% group_by(Type)
   %>% summarise(ct = n(), 
                 bh = sum(bh), 
                 p_bh = round(100*sum(bh)/ct,1),
                 by = sum(by), 
                 p_by = round(100*sum(by)/ct,1),
                 bf = sum(bf),
                 p_bf = round(100*sum(bf)/ct,1)) 
)

In [54]:
(step1 
    %>% group_by(Type, feature)
    %>% summarise( bh = sum(bh) > 0, by = sum(by) > 0, bf = sum(bf) > 0)
    %>% group_by(Type)
    %>% summarise(ct = n(), 
                  bh = sum(bh), 
                  p_bh = round(100*sum(bh)/ct,1),
                  by = sum(by), 
                  p_by = round(100*sum(by)/ct,1),
                  bf = sum(bf),
                  p_bf = round(100*sum(bf)/ct,1))
)

`summarise()` has grouped output by 'Type'. You can override using the `.groups` argument.


Type,ct,bh,p_bh,by,p_by,bf,p_bf
Somatic,16176,9188,56.8,1932,11.9,16,0.1
RNA: T-cell,519,297,57.2,126,24.3,13,2.5
RNA: TGFB,415,290,69.9,97,23.4,7,1.7
RNA: Proliferation,281,169,60.1,34,12.1,0,0.0
RNA: Remaining,7972,563,7.1,36,0.5,1,0.0
CNV/SVs,2500,28,1.1,0,0.0,0,0.0
Clinical,38,9,23.7,3,7.9,0,0.0
HLA,22,2,9.1,0,0.0,0,0.0
